EE 502 P: Analytical Methods for Electrical Engineering
# 8. Probability
## Prof. Eric Klavins, Electrical and Computer Engineering
### 25 September, 2019
Copyright &copy; 2019, University of Washington

Topics:
- Examples (coins, dice, waiting, Bolztman)
- Sample spaces
- PDFs and CDFs
- Random Variables
- Expected value, moments, and variances
- Joints and marginals
- Entropy and information
- Divergences and distances
- Markov chains and processes (and linear algebra)
- Markov decision processes